# additional explanation by mladen:
Some preface : in the attached article the author does a well job explaining what he felt a 2 most important points of Trend Quality indicators calculation. What is not explained there is how the "direction" (or "reversal" as the coders of tradestation, metastock, amibroker ... call it) is determined. It is a difference of 2 emas (7 and 15 ema by default) and since the difference of 2 emas is by definition macd, it turns out that the additional "spice" in these indicators is macd. It does not make these indicators a "yet another macd" indicator, since they are not that, but I felt it necessary to clarify that point too (to avoid some mis-expectations)

In [2]:
import pandas as pd
df = pd.read_csv('./data/av_eurusd_daily.csv')
df.set_index("timestamp", inplace=True)

In [3]:
df = df.head(365)

In [4]:
df

,open,high,low,close
timestamp,,,,
2023-11-16,1.08518,1.08955,1.08299,1.08572
2023-11-15,1.08788,1.08861,1.08313,1.08468
2023-11-14,1.06975,1.08875,1.06927,1.08787
2023-11-13,1.06826,1.07061,1.06647,1.06987
2023-11-10,1.06671,1.06930,1.06559,1.06848
...,...,...,...,...
2022-06-30,1.04407,1.04888,1.03810,1.04771
2022-06-29,1.05217,1.05355,1.04330,1.04390
2022-06-28,1.05845,1.06063,1.05020,1.05208


In [8]:
# calculating tq using the "pandas_ta" way

from pandas import concat, DataFrame, Series
import numpy as np
from pandas_ta import Imports
from pandas_ta.momentum import macd
from pandas_ta.utils import get_offset, verify_series, signals

def tq(close, trend_period=None, noise_period=None, correction_factor=None, fast_length=None, slow_length=None, talib=None, **kwargs):
  """Indicator: Trend Quality Q Indicator (TQ)"""
  # validate arguments
  trend_period = int(trend_period) if trend_period and trend_period > 0 else 4
  noise_period = int(noise_period) if noise_period and noise_period > 0 else 250
  correction_factor = int(correction_factor) if correction_factor and correction_factor > 0 else 2
  fast_length = int(fast_length) if fast_length and fast_length > 0 else 7
  slow_length = int(slow_length) if slow_length and slow_length > 0 else 14 
  close = verify_series(close, max(trend_period, noise_period, correction_factor, fast_length, slow_length))

  mode_tal = bool(talib) if isinstance(talib, bool) else True

  if close is None: return

  as_mode = kwargs.setdefault("asmode", False)

  # Calculcate Result

  # for now, not writing talib mode checks
  # if Imports["talib"] and mode_tal:

  # init df with close as passed close series of prices
  df = DataFrame(close, columns=['close'])
  # calculcate macd using pandas_ta
  calc_macd = macd(df['close'], fast=fast_length, slow=slow_length, talib=False)
  # extract the first column of the macd calculation
  df['macd'] = calc_macd.iloc[:,0]
  # calculate the sign of the macd (1 if positive, -1 if negative, 0 if 0)
  df['sign'] = np.sign(df['macd'])
  # set the tq column to 0
  df['cpc'] = 0
  df['trend'] = 0
  df['dt'] = 0
  df['tq'] = 0
  # resetting index here to access indexes as ints instead of datetime
  datetime = df.index
  df.reset_index(inplace=True)
  
  # iterate over df to calculate tq
  for row in df.itertuples():
    close = row.close
    macd_value = row.macd
    sign = row.sign
    cpc = row.cpc
    trend = row.trend
    dt = row.dt
    tq = row.tq
    i = row.Index

    if i == 0:
      prev_close = close
      close = 0
      prev_sign = sign
      continue
    
    prev_close = df.at[i-1, 'close'] 
    change = abs(close - prev_close) 
    
    if sign != prev_sign:
      pass
    else:
      cpc = sign * change + df.at[i-1, 'cpc']
      # trend is cpc smoothed
      trend = (cpc * (1/trend_period)) + (df.at[i-1, 'trend'] * (1 - (1/trend_period)))

    # In the next few lines, mladen calculates dt, avg_dt, 
    # and noise.  I'm not sure what that does so I'll stop here.
      
    # add something to tq
    # end of loop
  
  df.set_index(datetime, inplace=True)
  tq = df['tq']

  _asmode = "AS" if as_mode else ""
  # chose not to include noise length and correction factor in props
  _props = f"_{fast_length}_{slow_length}"
  tq.name = f"TQ{_asmode}{_props}"
  tq.category = "trend"

  data = {tq.name: tq}
  df = DataFrame(data)
  df.name = f"TQ{_asmode}{_props}"
  df.category = tq.category

  # for now, not implementing signal indicators
  
  return df

In [9]:
# return tq df without eurusd prices dataframe
tq(df['close'])

,TQ_7_14
timestamp,
2023-11-16,0
2023-11-15,0
2023-11-14,0
2023-11-13,0
2023-11-10,0
...,...
2022-06-30,0
2022-06-29,0
2022-06-28,0


In [7]:
# cpc approach usinf df.apply()
# this doesn't work that well since it returns a list inside each cell and prevents plotting
prev_macd_sign = None

def calculate_cpc(x):
    global prev_macd_sign
    if prev_macd_sign is not None and x['macd_sign'] != prev_macd_sign:
        cpc = 0
    else:
        cpc = x['close_diff'].cumsum()
    prev_macd_sign = x['macd_sign']
    return cpc

df['close_diff'] = df['close'].diff()
df['macd_sign'] = np.sign(df['macd'])

df['cpc'] = df.apply(calculate_cpc, axis=1)

In [7]:
# return eurusd prices with tq calculation
df['tq'] = tq(df['close'])
df


,open,high,low,close,tq
timestamp,,,,,
2023-11-16,1.08518,1.08955,1.08299,1.08572,NaN
2023-11-15,1.08788,1.08861,1.08313,1.08468,NaN
2023-11-14,1.06975,1.08875,1.06927,1.08787,NaN
2023-11-13,1.06826,1.07061,1.06647,1.06987,NaN
2023-11-10,1.06671,1.06930,1.06559,1.06848,NaN
...,...,...,...,...,...
2022-06-30,1.04407,1.04888,1.03810,1.04771,NaN
2022-06-29,1.05217,1.05355,1.04330,1.04390,NaN
2022-06-28,1.05845,1.06063,1.05020,1.05208,NaN
